In [ ]:
import pandas as pd
import optuna
import matplotlib.pyplot as plt

from calibrate import read_data
from plot import to_df, get_pareto_front, get_contour_plot, get_parallel_coordinate_plot

%matplotlib inline

In [ ]:
storage = "sqlite:///calibration.db"
print("Studies:")
for s in optuna.study.get_all_study_summaries(storage):
    print(s.study_name, s.datetime_start, s.n_trials, "trials")

In [ ]:
study = optuna.multi_objective.load_study("multi", storage)

In [ ]:
unc = optuna.load_study("unconstrained", storage)
uf = to_df(unc)
print(uf.loc[uf.value.idxmin()])
optuna.visualization.plot_parallel_coordinate(unc)

In [ ]:
tf, fig = get_pareto_front(study, hover=["alpha", "ciCorrection"], color="ciCorrection")
fig.show()

In [ ]:
number = 594

start="2020-03-01"
end="2020-04-20"
#f = "output-calibration/%d/infections.txt" % number
f = "output-compare/infections.txt"

df, hospital, rki = read_data(f, "Berlin", "berlin-hospital.csv", "berlin-cases.csv")

df = df[(df.date >= start) & (df.date <= end)]
hospital = hospital[(hospital.Datum >= start) & (hospital.Datum <= end)]
rki = rki[(rki.date >= start) & (rki.date <= end)]

tf[tf.number == number]

In [ ]:
ax = df.plot(x="date", y=["cases"], label=["Simulated Cases"], logy=True, figsize=(16, 8))
rki.plot(x="date", y=["cases"], label=["RKI Cases"], ax=ax)
plt.title("Cases")

#dz = df.cases.sum() / rki.cases.sum()
dz = float(df.nContagiousCumulative.tail(1) / rki.casesCumulative.tail(1))

print("Dunkelziffer", dz , "=", (100 - 100/dz), "%")

In [ ]:
ax = df.plot(x="date", y=["casesNorm"], label=["Sim. Cases norm."], figsize=(16, 8))
rki.plot(x="date", y=["casesNorm"], label=["RKI cases norm"], ax=ax)
plt.title("Normalized cases")

In [ ]:
ax = df.plot(x="date", y=["nSeriouslySick", "nCritical"], logy=True, figsize=(16, 8))
hospital.plot(x="Datum", y=["Stationäre Behandlung", "Intensivmedizin"], logy=True, ax=ax)

plt.title("Hospitalization")

In [ ]:
get_parallel_coordinate_plot(study)

In [ ]:
get_contour_plot(study, "alpha", "ciCorrection")